# Convolutional Neural Network

### Importing the libraries

In [265]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.preprocessing.image import ImageDataGenerator

In [266]:
tf.__version__

'2.9.1'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [267]:
train_datagen = ImageDataGenerator(rescale = 1./255,
#                                    shear_range = 0.2,
#                                    zoom_range = 0.2,
#                                    horizontal_flip = True
                                  rotation_range=20,           # Rotate the image by up to 20 degrees
    width_shift_range=0.2,       # Shift the image horizontally by up to 20% of the image width
    height_shift_range=0.2,      # Shift the image vertically by up to 20% of the image height
    shear_range=0.2,             # Shear the image by up to 20 degrees
    zoom_range=0.2,              # Zoom in or out on the image by up to 20%
    horizontal_flip=True,        # Flip the image horizontally
    fill_mode='nearest'
                                  
                                  )


training_set = train_datagen.flow_from_directory('trainset\patches_in_throat',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 15 images belonging to 3 classes.


### Preprocessing the Test set

In [268]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('testset\patches_in_throat',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 3 images belonging to 3 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [269]:
cnn = tf.keras.models.Sequential()



### Step 1 - Convolution

In [270]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

### Step 2 - Pooling

In [271]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [272]:
cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [273]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [274]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Step 5 - Output Layer

In [275]:
cnn.add(tf.keras.layers.Dense(units=3, activation='softmax'))

## Part 3 - Training the CNN

### Compiling the CNN

In [276]:
cnn.compile(loss = 'categorical_crossentropy', optimizer = 'adam', 
              metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [277]:
import os

dir_path = r'C:\Users\Geetha\Desktop\std_severity\trainset\patches_in_throat'

# Get a list of all the files in the directory
file_list = os.listdir(dir_path)

# Loop through the files and delete any with the ".ini" extension
for file_name in file_list:
    if file_name.endswith('.ini'):
        file_path = os.path.join(dir_path, file_name)
        os.remove(file_path)
#os.remove(r'C:\Users\Geetha\Desktop\STD_CNN\testset\muscle_wasting\desktop.ini')
#os.remove(r'C:\Users\Geetha\Desktop\STD_CNN\testset\yellowing_skin\desktop.ini')


In [278]:
import os
import cv2
bad_list=[]
dir=r'C:\Users\Geetha\Desktop\std_severity\trainset\patches_in_throat'
subdir_list=os.listdir(dir)
for d in subdir_list:  
    dpath=os.path.join (dir, d) 
    if d in ['testset', 'trainset']:
        class_list=os.listdir(dpath) 
       # print (class_list)
        for klass in class_list: 
            class_path=os.path.join(dpath, klass) 
            #print(class_path)
            file_list=os.listdir(class_path) 
            for f in file_list: 
                fpath=os.path.join (class_path,f)
                index=f.rfind('.') 
                ext=f[index+1:] 
                if ext  not in ['jpg', 'png', 'bmp', 'gif']:
                    print(f'file {fpath}  has an invalid extension {ext}')
                    bad_list.append(fpath)                    
                else:
                    try:
                        img=cv2.imread(fpath)
                        size=img.shape
                    except:
                        print(f'file {fpath} is not a valid image file ')
                        bad_list.append(fpath)
                       
print (bad_list)

[]


In [279]:
cnn.fit(x = training_set,validation_data=test_set,  epochs = 25)


Epoch 1/25
1/1 [==============================] - 1s 1s/step - loss: 1.0991 - accuracy: 0.2667 - val_loss: 3.4332 - val_accuracy: 0.3333
Epoch 2/25
1/1 [==============================] - 0s 268ms/step - loss: 2.1281 - accuracy: 0.5333 - val_loss: 1.7971 - val_accuracy: 0.3333
Epoch 3/25
1/1 [==============================] - 0s 251ms/step - loss: 1.1113 - accuracy: 0.5333 - val_loss: 1.2449 - val_accuracy: 0.3333
Epoch 4/25
1/1 [==============================] - 0s 246ms/step - loss: 1.0916 - accuracy: 0.2667 - val_loss: 1.1672 - val_accuracy: 0.3333
Epoch 5/25
1/1 [==============================] - 0s 258ms/step - loss: 1.2034 - accuracy: 0.2667 - val_loss: 1.1185 - val_accuracy: 0.3333
Epoch 6/25
1/1 [==============================] - 0s 240ms/step - loss: 1.1643 - accuracy: 0.2667 - val_loss: 1.0999 - val_accuracy: 0.3333
Epoch 7/25
1/1 [==============================] - 0s 253ms/step - loss: 1.1145 - accuracy: 0.2667 - val_loss: 1.1056 - val_accuracy: 0.6667
Epoch 8/25
1/1 [=======

In [280]:

class_indices = training_set.class_indices
print(class_indices)

# Get the class label for a particular class




{'discoloured': 0, 'not_very_severe': 1, 'white_patches': 2}


## Part 4 - Making a single prediction

In [301]:
import numpy as np
from tensorflow.keras.utils import load_img, img_to_array
from keras.preprocessing import image
test_image = tf.keras.utils.load_img('single_prediction/pwhite_patches2.jpg', target_size = (64, 64))
test_image = tf.keras.preprocessing.image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
print(result)
training_set.class_indices
if result[0][0] >result[0][1] and result[0][0] >result[0][2]:
  prediction = 'discoloured'
elif result[0][1] >result[0][0] and result[0][1] >result[0][2]:
  prediction = 'not very severe'
elif result[0][2] >result[0][0] and result[0][2] >result[0][1]:
  prediction = 'white patches'


probs = cnn.predict(test_image)
print(probs)

prob_sum = np.sum(probs)
print(prob_sum)

indices=np.argsort(probs)
print(indices)
print(probs[0][indices])
if probs[0][indices[0][0]]> probs[0][indices[0][1]]:
    print("a is larger than b")
else:
    print("b is larger than a")
    
top_3 = np.argsort(probs,axis=1)[:, -3:]
print(top_3)
class_probs = np.take_along_axis(probs, top_3, axis=1)
print(class_probs)

print(f"The test sample {test_image} ")
print("Top three Predicted classes and their corresponding probabilties\n")
for j in range(3):
        class_idx = top_3[0][2-j]
        class_prob = class_probs[0][2-j]
        print(f"\tClass {class_idx}: {class_prob:.5f}")
# print(f"Actual class to which the test sample belongs to {}  ")�


1/1 [==============================] - 0s 21ms/step
[[0.         0.00418815 0.9958118 ]]
1/1 [==============================] - 0s 20ms/step
[[0.         0.00418815 0.9958118 ]]
1.0
[[0 1 2]]
[[0.         0.00418815 0.9958118 ]]
b is larger than a
[[0 1 2]]
[[0.         0.00418815 0.9958118 ]]
The test sample [[[[ 94.   0.   0.]
   [ 95.   0.   0.]
   [ 92.   1.   0.]
   ...
   [226.  77.  79.]
   [233.  87.  90.]
   [253.  99. 107.]]

  [[101.   1.   3.]
   [103.   0.   2.]
   [ 82.   1.   0.]
   ...
   [187.  50.  58.]
   [209.  66.  70.]
   [234.  87.  93.]]

  [[ 96.   0.   2.]
   [ 94.   0.   1.]
   [ 80.   2.   0.]
   ...
   [149.  26.  29.]
   [189.  48.  56.]
   [211.  71.  80.]]

  ...

  [[185.  80.  85.]
   [190.  96.  96.]
   [211. 122. 126.]
   ...
   [228.  60.  73.]
   [232.  58.  68.]
   [231.  68.  71.]]

  [[198.  72.  73.]
   [206.  91.  94.]
   [212. 120. 121.]
   ...
   [234.  59.  74.]
   [231.  66.  72.]
   [234.  66.  79.]]

  [[215.  65.  77.]
   [207.  71.  73

In [302]:
print(prediction)

white patches


In [303]:
print(result)

[[0.         0.00418815 0.9958118 ]]
